In [1]:
import os
import sys

import pandas as pd

root_path = os.path.abspath(os.path.join(os.getcwd(), '..'))
sys.path.append(root_path)

In [2]:
from core.backtesting.optimizer import StrategyOptimizer

optimizer = StrategyOptimizer(root_path=root_path, database_name="optimization_database")
optimizer.get_all_study_names()

['macd_bb_v1_with_optimizer',
 'pmm_simple_with_optimizer',
 'smugplug_v1_with_optimizer']

In [3]:
df = optimizer.get_study_trials_df("smugplug_v1_with_optimizer")
df

,number,value,datetime_start,datetime_complete,duration,params_atr_length,params_atr_multiplier,params_long_ema,params_macd_fast,params_macd_signal,...,profit_factor,sharpe_ratio,total_executors,total_executors_with_position,total_long,total_positions,total_short,total_volume,win_signals,state
100,100,0.33876251,2024-11-15 14:15:06.614734,2024-11-15 14:16:24.081812,0 days 00:01:17.467078,81,1.13723399,101,46,53,...,1.17062012,0.33876251,630,630,393,630,237,420000,477,COMPLETE
101,101,0.99370292,2024-11-15 14:16:24.096486,2024-11-15 14:17:27.748611,0 days 00:01:03.652125,84,1.08140688,96,46,53,...,1.22799968,0.99370292,646,646,403,646,243,430666.66666667,495,COMPLETE
102,102,1.16932268,2024-11-15 14:17:27.774125,2024-11-15 14:18:39.258183,0 days 00:01:11.484058,94,1.08456199,96,46,53,...,1.22924013,1.16932268,652,652,404,652,248,434666.66666667,500,COMPLETE
103,103,0.89682927,2024-11-15 14:18:39.277495,2024-11-15 14:19:51.500521,0 days 00:01:12.223026,96,1.08460232,97,46,53,...,1.21989063,0.89682927,638,638,396,638,242,425333.33333333,488,COMPLETE
104,104,0.57035283,2024-11-15 14:19:51.533628,2024-11-15 14:20:57.780109,0 days 00:01:06.246481,80,1.13667998,96,46,55,...,1.27549992,0.57035283,597,597,375,597,222,398000,460,COMPLETE
105,105,0.89604517,2024-11-15 14:20:57.791910,2024-11-15 14:22:10.485970,0 days 00:01:12.694060,81,1.07874377,97,46,53,...,1.21822419,0.89604517,637,637,395,637,242,424666.66666667,487,COMPLETE


In [4]:
df_filtered = df[df["value"] > 0].sort_values("total_executors", ascending=False)
df_filtered

,number,value,datetime_start,datetime_complete,duration,params_atr_length,params_atr_multiplier,params_long_ema,params_macd_fast,params_macd_signal,...,profit_factor,sharpe_ratio,total_executors,total_executors_with_position,total_long,total_positions,total_short,total_volume,win_signals,state
102,102,1.16932268,2024-11-15 14:17:27.774125,2024-11-15 14:18:39.258183,0 days 00:01:11.484058,94,1.08456199,96,46,53,...,1.22924013,1.16932268,652,652,404,652,248,434666.66666667,500,COMPLETE
101,101,0.99370292,2024-11-15 14:16:24.096486,2024-11-15 14:17:27.748611,0 days 00:01:03.652125,84,1.08140688,96,46,53,...,1.22799968,0.99370292,646,646,403,646,243,430666.66666667,495,COMPLETE
103,103,0.89682927,2024-11-15 14:18:39.277495,2024-11-15 14:19:51.500521,0 days 00:01:12.223026,96,1.08460232,97,46,53,...,1.21989063,0.89682927,638,638,396,638,242,425333.33333333,488,COMPLETE
105,105,0.89604517,2024-11-15 14:20:57.791910,2024-11-15 14:22:10.485970,0 days 00:01:12.694060,81,1.07874377,97,46,53,...,1.21822419,0.89604517,637,637,395,637,242,424666.66666667,487,COMPLETE
100,100,0.33876251,2024-11-15 14:15:06.614734,2024-11-15 14:16:24.081812,0 days 00:01:17.467078,81,1.13723399,101,46,53,...,1.17062012,0.33876251,630,630,393,630,237,420000,477,COMPLETE
104,104,0.57035283,2024-11-15 14:19:51.533628,2024-11-15 14:20:57.780109,0 days 00:01:06.246481,80,1.13667998,96,46,55,...,1.27549992,0.57035283,597,597,375,597,222,398000,460,COMPLETE


In [7]:
import plotly.express as px

px.scatter(df_filtered, x="total_executors", y="sharpe_ratio", color="net_pnl", hover_data=["number"])

In [9]:
from controllers.directional_trading.xtreet_bb import XtreetBBControllerConfig
from core.backtesting import BacktestingEngine
import json

import datetime

id_to_backtest = 102
resolution = "1m"
config = json.loads(df[df["number"] == id_to_backtest].iloc[0]["config"])
connector_name = config["connector_name"]
trading_pair = config["trading_pair"]
BACKTESTING_DAYS = 30
start_date = datetime.datetime.now() - datetime.timedelta(days=BACKTESTING_DAYS)
end_date = datetime.datetime.now()

backtesting_engine = BacktestingEngine(root_path=root_path, load_cached_data=False)
config = backtesting_engine.get_controller_config_instance_from_dict(config)

In [10]:
bt_result = await backtesting_engine.run_backtesting(config=config, start=int(start_date.timestamp()), end=int(end_date.timestamp()), backtesting_resolution=resolution)

2024-11-15 14:24:03,655 - asyncio - ERROR - Unclosed client session
client_session: <aiohttp.client.ClientSession object at 0x3148d0340>
2024-11-15 14:24:03,657 - asyncio - ERROR - Unclosed connector
connections: ['[(<aiohttp.client_proto.ResponseHandler object at 0x310ae1360>, 196380.756222375)]']
connector: <aiohttp.connector.TCPConnector object at 0x3148d03d0>
2024-11-15 14:24:22,954 - asyncio - ERROR - Unclosed client session
client_session: <aiohttp.client.ClientSession object at 0x31524c670>
2024-11-15 14:24:22,964 - asyncio - ERROR - Unclosed connector
connections: ['[(<aiohttp.client_proto.ResponseHandler object at 0x3151fe980>, 196400.043556291)]']
connector: <aiohttp.connector.TCPConnector object at 0x31524c6a0>
2024-11-15 14:24:23,659 - asyncio - ERROR - Unclosed client session
client_session: <aiohttp.client.ClientSession object at 0x31524cc40>
2024-11-15 14:24:23,660 - asyncio - ERROR - Unclosed connector
connections: ['[(<aiohttp.client_proto.ResponseHandler object at 0x3

In [11]:
print(bt_result.get_results_summary())
bt_result.get_backtesting_figure()


Net PNL: $432.43 (43.24%) | Max Drawdown: $-91.55 (-9.07%)
Total Volume ($): 289333.33 | Sharpe Ratio: 1.27 | Profit Factor: 1.46
Total Executors: 434 | Accuracy Long: 0.79 | Accuracy Short: 0.77
Close Types: Take Profit: 7 | Stop Loss: 92 | Time Limit: 0 |
             Trailing Stop: 335 | Early Stop: 0



In [ ]:
from core.services.backend_api_client import BackendAPIClient

client = BackendAPIClient()

In [ ]:
bt_result.controller_config.dict()

In [ ]:
await client.add_controller_config(json.loads(bt_result.controller_config.json()))

In [ ]:
await client.deploy_script_with_controllers(
    bot_name="test_1",
    controller_configs=[bt_result.controller_config.id + ".yml"],
    image_name="dardonacci/hummingbot:latest",
    max_global_drawdown=50,
    max_controller_drawdown=20,
)